In [1]:
import pandas as pd

trade = pd.read_csv('../FinalProject_old/Trade_DetailedTradeMatrix.csv', encoding = "ISO-8859-1") 


In [22]:
import numpy as np
region = pd.read_csv('../FinalProject_old/countries.csv', encoding = "ISO-8859-1") 
region.head(5)

,CountryName,CountryCode,ContinentCode,CenterLongitude,CenterLatitude,Population
0,Afghanistan,AF,AS,65.0,33.00,32526560
1,Albania,AL,EU,20.0,41.00,2896680
2,Algeria,DZ,AF,3.0,28.00,39666520
3,American Samoa,AS,OC,-170.5,-14.32,55540
4,Andorra,AD,EU,1.5,42.50,70470


In [23]:
region_sub = region[['CountryName', 'ContinentCode']]

In [125]:
pop = pd.read_csv('Population.csv', encoding = "ISO-8859-1")
pop_sub = pop[pop['Element']=='Total Population - Both sexes']
pop_sub=pop_sub[['Area','Year Code','Value']]
pop_sub.head(10)

,Area,Year Code,Value
0,Afghanistan,1950,7752.118
1,Afghanistan,1951,7839.510
2,Afghanistan,1952,7934.980
3,Afghanistan,1953,8038.596
4,Afghanistan,1954,8150.447
5,Afghanistan,1955,8270.581
6,Afghanistan,1956,8399.030
7,Afghanistan,1957,8535.807
8,Afghanistan,1958,8680.946
9,Afghanistan,1959,8834.445


In [147]:
 
pop_sub.columns =['Reporter Countries','Year Code', 'population']
pop_sub["Year Code"]= pop_sub["Year Code"].astype(str)
pop_sub["population"] = (pop_sub["population"]*1000).astype(int)

In [3]:
## for force  chart 
trade_group = trade.groupby(['Reporter Countries', 'Partner Countries', 'Element','Year Code']).agg({'Value':np.sum})

In [168]:
trade.head(20)

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,9,Argentina,1232,Food prep nes,5610,Import Quantity,2011,2011,tonnes,1.0,NaN
1,2,Afghanistan,9,Argentina,1232,Food prep nes,5622,Import Value,2011,2011,1000 US$,4.0,NaN
2,2,Afghanistan,9,Argentina,1058,"Meat, chicken",5610,Import Quantity,2011,2011,tonnes,198.0,NaN
3,2,Afghanistan,9,Argentina,1058,"Meat, chicken",5622,Import Value,2011,2011,1000 US$,281.0,NaN
4,2,Afghanistan,9,Argentina,30,Rice - total (Rice milled equivalent),5610,Import Quantity,2016,2016,tonnes,0.0,A
5,2,Afghanistan,9,Argentina,30,Rice - total (Rice milled equivalent),5622,Import Value,2016,2016,1000 US$,0.0,A
6,2,Afghanistan,9,Argentina,164,Sugar refined,5610,Import Quantity,2011,2011,tonnes,315.0,NaN
7,2,Afghanistan,9,Argentina,164,Sugar refined,5622,Import Value,2011,2011,1000 US$,130.0,NaN
8,2,Afghanistan,1,Armenia,828,Cigarettes,5610,Import Quantity,2011,2011,tonnes,8.0,NaN
9,2,Afghanistan,1,Armenia,828,Cigarettes,5622,Import Value,2011,2011,1000 US$,48.0,NaN


In [13]:
trade_total = trade_group.reset_index()
trade_total = trade_total[trade_total['Value']!=0]
trade_import = trade_total[(trade_total['Element'] =='Import Value') & (trade_total['Year Code']==2016)]
trade_export = trade_total[(trade_total['Element'] =='Export Value') & (trade_total['Year Code']==2016)]

In [16]:
trade_import.head(5)
print(len(trade_import))

13932


In [17]:
### for directed force graph 
trade_export.columns=['source','target','Element','Year Code','value']
trade_import.columns=['target','source','Element','Year Code','value']
export_out = trade_export[['source','target','value']]
import_out = trade_import[['source','target','value']]
print(len(import_out))
import_out = import_out.append(export_out, ignore_index=True)
print(len(import_out))

13932
27261


In [41]:
import_out_group = import_out.sort_values(by=['source','value'], ascending=False).groupby('source').head(1)
import_out_group.head(5)

,source,target,value
27250,Zimbabwe,South Africa,901404.0
13931,Zambia,Zimbabwe,130423.0
9572,Yemen,Oman,9119.0
9715,Western Sahara,Pakistan,7544.0
2979,Viet Nam,"China, mainland",2739226.0


In [42]:
import_out_group.to_csv('countryrelation_top1.csv',index=False)

In [25]:
 
import_out_region = pd.merge(import_out, region_sub, how='inner', left_on=['source'], right_on=['CountryName'])
import_out_region.head(5)

,source,target,value,CountryName,ContinentCode
0,India,Afghanistan,418.0,India,AS
1,India,Albania,6201.0,India,AS
2,India,Algeria,218884.0,India,AS
3,India,Antigua and Barbuda,18.0,India,AS
4,India,Argentina,17986.0,India,AS


In [26]:
import_out_region1 = import_out_region[['source','target','value','ContinentCode']]
import_out_region1.columns=['source','target','value','sourcecont']
import_out_region2 = pd.merge(import_out_region1, region_sub, how='inner', left_on=['target'], right_on=['CountryName'])
import_out_region2.head(5)

,source,target,value,sourcecont,CountryName,ContinentCode
0,India,Afghanistan,418.0,AS,Afghanistan,AS
1,India,Afghanistan,63094.0,AS,Afghanistan,AS
2,Kazakhstan,Afghanistan,64.0,AS,Afghanistan,AS
3,Kazakhstan,Afghanistan,408308.0,AS,Afghanistan,AS
4,Kyrgyzstan,Afghanistan,2.0,AS,Afghanistan,AS


In [31]:
import_out_region3 = import_out_region2[['source','target','value','sourcecont','ContinentCode']]
import_out_region3.columns=['source','target','value','sourcecont','targetcont']
AS = import_out_region3[(import_out_region3['sourcecont']=='AS')]
AF = import_out_region3[(import_out_region3['sourcecont']=='AF')]
EU = import_out_region3[(import_out_region3['sourcecont']=='EU') ]
NA = import_out_region3[(import_out_region3['sourcecont']=='NA') ]
SA = import_out_region3[(import_out_region3['sourcecont']=='SA') ]
OC = import_out_region3[(import_out_region3['sourcecont']=='OC') ]
AF.head(5)
OC.head(5)

,source,target,value,sourcecont,targetcont
43,Australia,Afghanistan,659.0,OC,AS
48,New Zealand,Afghanistan,406.0,OC,AS
159,Australia,Albania,1057.0,OC,EU
164,New Zealand,Albania,125.0,OC,EU
269,Australia,Algeria,6270.0,OC,AF


In [32]:
AS.to_csv('countryrelation_AS.csv',index=False)
AF.to_csv('countryrelation_AF.csv',index=False)
NA.to_csv('countryrelation_NA.csv',index=False)
SA.to_csv('countryrelation_SA.csv',index=False)
EU.to_csv('countryrelation_EU.csv',index=False)
OC.to_csv('countryrelation_OC.csv',index=False)

In [34]:
china = import_out[(import_out['source']=='China, mainland') | (import_out['target']=='China, mainland')]
china.to_csv('countryrelation_china.csv',index=False)
#trade_import.to_csv('trade_import.csv')
#trade_export.to_csv('trade_export.csv')

In [36]:
#trade_import = pd.read_csv('trade_import.csv')
#trade_import.head(5)

,Unnamed: 0,Reporter Countries,Partner Countries,Element,Year Code,Value
0,197,Afghanistan,India,Import Quantity,2016,579.0
1,280,Afghanistan,Kazakhstan,Import Quantity,2016,399.0
2,298,Afghanistan,Kyrgyzstan,Import Quantity,2016,2.0
3,333,Afghanistan,Netherlands,Import Quantity,2016,12.0
4,361,Afghanistan,Pakistan,Import Quantity,2016,326690.0


In [40]:
trade_import_sub = trade_import[['Reporter Countries','Partner Countries','Value']]
trade_export_sub = trade_export[['Reporter Countries','Partner Countries','Value']]

In [41]:
#### for both name and value only 
def mydic(df, w, q):
    namelist = []
    df = df.reset_index()
     
    for i in range(len(df)):
        namedic = dict([('name',df.loc[i,'Partner Countries']), ('value',df.loc[i,'Value'])])
        namelist.append(namedic)
    return namelist
   
trade_import_val = trade_import_sub.groupby('Reporter Countries').apply(mydic, 'Partner Countries','Value')

trade_import_val = trade_import_val.reset_index()
trade_import_val.columns=['name','children']
trade_import_val.head(5)

trade_export_val = trade_export_sub.groupby('Reporter Countries').apply(mydic, 'Partner Countries','Value')
trade_export_val = trade_export_val.reset_index()
trade_export_val.columns=['name','children']
trade_export_val.head(5)

mylist=[]
for i in range(len(trade_import_val)):
    mylist.append({'name': trade_import_val.loc[i,'name'], 'children': trade_import_val.loc[i,'children']})
import json
with open('trade_imports_2011.json', 'w', encoding='utf-8') as fout:
    json.dump(mylist , fout, ensure_ascii=False, indent=4)
    
mylist_ex=[]
for i in range(len(trade_export_val)):
    mylist_ex.append({'name': trade_export_val.loc[i,'name'], 'children': trade_export_val.loc[i,'children']})
import json
with open('trade_exports_2016.json', 'w', encoding='utf-8') as fout_ex:
    #json.dump(mylist_ex , fout_ex, ensure_ascii=False, indent=4)
    json.dump(mylist_ex , fout_ex, ensure_ascii=False)
    

In [30]:
## for bubble chart 
trade_group = trade.groupby(['Reporter Countries', 'Element','Year Code']).agg({'Value':np.sum})
trade_total = trade_group.reset_index()
trade_import = trade_total[(trade_total['Element'] =='Import Value')]
trade_export = trade_total[(trade_total['Element'] =='Export Value')]

In [153]:
datelist = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]
date_seq = pd.DataFrame(datelist, columns=['Year Code'])
trade_import_group = trade_import[['Reporter Countries','Year Code','Value']].groupby(['Reporter Countries'])
data_comb = pd.DataFrame()
for name, group in trade_import_group:
    date_seq['Reporter Countries']  = name
    date_seq['Value'] = 0
    temp = pd.DataFrame()
    temp = temp.append(group)
    temp = temp.append(date_seq)
     
    temp.drop_duplicates(subset = 'Year Code', keep='first', inplace=True)
    temp = temp.sort_values(by='Year Code', ascending=False)
    temp = temp.head(10)
    temp =  temp.sort_values(by='Year Code') 
    temp["Year Code"]= temp["Year Code"].astype(str)
     
    temp["Value"]= temp["Value"]
    
    data_comb = data_comb.append(temp)
data_comb.head(10)



,Reporter Countries,Value,Year Code
0,Afghanistan,0.0,2007
1,Afghanistan,0.0,2008
18,Afghanistan,624317.0,2009
19,Afghanistan,681095.0,2010
20,Afghanistan,846878.0,2011
5,Afghanistan,0.0,2012
6,Afghanistan,0.0,2013
21,Afghanistan,100948.0,2014
22,Afghanistan,156757.0,2015
23,Afghanistan,134452.0,2016


In [154]:

trade_export_group = trade_export[['Reporter Countries','Year Code','Value']].groupby(['Reporter Countries'])

 
data_comb_ex = pd.DataFrame()
for name, group in trade_export_group:
    date_seq['Reporter Countries']  = name
    date_seq['Value'] = int(0)
    temp = pd.DataFrame()
    temp = temp.append(group)
    temp = temp.append(date_seq)
     
    temp.drop_duplicates(subset = 'Year Code', keep='first', inplace=True)
    temp = temp.sort_values(by='Year Code', ascending=False)
    temp = temp.head(10)
    temp = temp.sort_values(by='Year Code')
    temp["Year Code"]= temp["Year Code"].astype(str)
    temp["Value"]= temp["Value"] 
    #temp = temp.astype({'Year Code': 'string'})
    data_comb_ex = data_comb_ex.append(temp)
data_comb_ex.head(10)


,Reporter Countries,Value,Year Code
0,Afghanistan,0.0,2007
1,Afghanistan,0.0,2008
6,Afghanistan,282049.0,2009
7,Afghanistan,268673.0,2010
8,Afghanistan,235164.0,2011
5,Afghanistan,0.0,2012
6,Afghanistan,0.0,2013
9,Afghanistan,0.0,2014
10,Afghanistan,21.0,2015
11,Afghanistan,54.0,2016


In [155]:
data_comb.columns=['Reporter Countries','importvalue', 'Year Code'] 
data_comb_ex.columns=['Reporter Countries','exportvalue', 'Year Code'] 
comb = pd.merge(data_comb, data_comb_ex, how='inner', left_on=['Reporter Countries', 'Year Code'], right_on=['Reporter Countries', 'Year Code'])
comb.head(10)

,Reporter Countries,importvalue,Year Code,exportvalue
0,Afghanistan,0.0,2007,0.0
1,Afghanistan,0.0,2008,0.0
2,Afghanistan,624317.0,2009,282049.0
3,Afghanistan,681095.0,2010,268673.0
4,Afghanistan,846878.0,2011,235164.0
5,Afghanistan,0.0,2012,0.0
6,Afghanistan,0.0,2013,0.0
7,Afghanistan,100948.0,2014,0.0
8,Afghanistan,156757.0,2015,21.0
9,Afghanistan,134452.0,2016,54.0


In [166]:
comb_pop = pd.merge(comb, pop_sub, how = 'left', left_on=['Reporter Countries', 'Year Code'], right_on=['Reporter Countries', 'Year Code']) 
comb_pop.head(10)

,Reporter Countries,importvalue,Year Code,exportvalue,population
0,Afghanistan,0.0,2007,0.0,2661000.0
1,Afghanistan,0.0,2008,0.0,2729000.0
2,Afghanistan,624317.0,2009,282049.0,2800000.0
3,Afghanistan,681095.0,2010,268673.0,2880000.0
4,Afghanistan,846878.0,2011,235164.0,2970000.0
5,Afghanistan,0.0,2012,0.0,3069000.0
6,Afghanistan,0.0,2013,0.0,3173000.0
7,Afghanistan,100948.0,2014,0.0,3275000.0
8,Afghanistan,156757.0,2015,21.0,3373000.0
9,Afghanistan,134452.0,2016,54.0,3465000.0


In [167]:
#### for both name and value only bubble chart
 
def mydic(df, w, q, p, x):
    namelist = []
    exlist = []
    poplist = []
    df = df.reset_index()
     
    for i in range(len(df)):
        namedic1 = [df.loc[i,'Year Code'], df.loc[i,'importvalue']]
        namedic2 = [df.loc[i,'Year Code'], df.loc[i,'exportvalue']]
        namedic3 = [df.loc[i,'Year Code'], df.loc[i,'population']]
        namelist.append(namedic1)
        exlist.append(namedic2)
        poplist.append(namedic3)
    return namelist, exlist, poplist

 
trade_export_val = comb_pop.groupby('Reporter Countries').apply(mydic, 'Year Code','importvalue', 'exportvalue','population')
trade_export_val = trade_export_val.reset_index()
trade_export_val.columns=['name','value']
trade_export_val.head(5)

mylist_ex=[]
for i in range(len(trade_export_val)):
    mylist_ex.append({'name': trade_export_val.loc[i,'name'], 'importvalue': trade_export_val.loc[i,'value'][0],
                     'exportvalue': trade_export_val.loc[i,'value'][1], 'population': trade_export_val.loc[i,'value'][2]})
import json

with open('trade_exports_value.json', 'w') as outfile:
    outfile.write('[')
    for entry in mylist_ex:
        json.dump(entry, outfile)
        outfile.write(',')
        outfile.write('\n')
    outfile.write(']')
        
#with open('trade_exports_value.json', 'w', encoding='utf-8') as fout_ex:
    #json.dump(mylist_ex , fout_ex, ensure_ascii=False, indent=4)
#    json.dump(mylist_ex , fout_ex, ensure_ascii=False)

In [89]:
trade_import_val = data_comb.groupby('Reporter Countries').apply(mydic, 'Year Code','Value')
trade_import_val.head(5)
trade_import_val = trade_import_val.reset_index()
trade_import_val.columns=['name','value']
 
trade_import_val.head(5)

mylist=[]
 
for i in range(len(trade_import_val)):
    
    mylist.append({"name": trade_import_val.loc[i,'name'], "value": trade_import_val.loc[i,'value']})
     

print(mylist[2])
import json
with open('trade_imports_value.json', 'w', encoding='utf-8') as fout:
    json.dump(mylist , fout, ensure_ascii=False)
    


{'name': 'Algeria', 'value': [['2007', 5364383.0], ['2008', 8268951.0], ['2009', 6270834.0], ['2010', 0.0], ['2011', 10462910.0], ['2012', 9739826.0], ['2013', 10160930.0], ['2014', 11386383.0], ['2015', 9694067.0], ['2016', 8649392.0]]}


In [16]:
###### for country name only 
def mydic(mylist):
    namelist = []
    for i in mylist:
        namedic = dict([('name', i)])
        namelist.append(namedic)
     
    return namelist
trade_import_dic = trade_import.groupby('Reporter Countries')['Partner Countries'].apply(mydic)
trade_import_dic.head(5)
trade_import_dic = trade_import_dic.reset_index()
trade_import_dic.columns=['name','children']
#### for value only
def mydic(mylist):
    namelist = []
    for i in mylist:
        namedic = dict([('value', i)])
        namelist.append(namedic)
     
    return namelist
trade_import_val = trade_import.groupby('Reporter Countries')['Value'].apply(mydic)
trade_import_val.head(5)